In [1]:
from bs4 import BeautifulSoup
import requests
import random
import time
import numpy as np
import pandas as pd
import re

In [7]:
from selenium import webdriver
browser = webdriver.Firefox()

In [3]:
browser.get('https://1xstavka.ru/line/ufc/2503366-ufc-287')
browser.implicitly_wait(2)
html = browser.page_source


# browser.quit()

In [4]:
bsObj = BeautifulSoup(html, 'lxml')
event_inf_tags = bsObj.findAll('div',{'class':'c-events__item c-events__item_game'})


In [5]:
names = np.array([[re.sub(r'\(.*\)', '' , item.get_text()).strip() for item in event.find_all('span',{'class':'c-events__team'})] for event in event_inf_tags])
names

array([['Крис Барнетт', 'Чейс Шерман'],
       ['Джеральд Миршерт', 'Джозеф Пайфер'],
       ['Игнасио Багамондес', 'Николас Мотта'],
       ['Келвин Гастелум', 'Крис Кертис'],
       ['Стив Гарсия-младший', 'Шайлан Нурданбек'],
       ['Сэм Хьюз', 'Жаклин Аморим'],
       ['Мишель Уотерсон', 'Луана Пиньейро'],
       ['Синтия Кальвильо', 'Люпита Годинез'],
       ['Роб Фонт', 'Адриан Янез'],
       ['Кевин Холланд', 'Сантьяго Понзиниббио'],
       ['Гилберт Бернс', 'Хорхе Масвидал'],
       ['Алекс Перейра', 'Исраэль Адесаня']], dtype='<U20')

In [6]:
coefs = [[item.get_text().strip() for item in event.find_all('span',{'class':'c-bets__bet c-bets__bet_coef c-bets__bet_sm'})] for event in event_inf_tags]
coefs = np.array([[it[0], it[2]]  if len(it)>2 else [0,0] for it in coefs])
coefs

array([['2.748', '1.45'],
       ['2.464', '1.54'],
       ['1.372', '3.015'],
       ['1.83', '1.94'],
       ['2.69', '1.46'],
       ['2.976', '1.39'],
       ['2.43', '1.552'],
       ['3.14', '1.37'],
       ['2.348', '1.59'],
       ['1.39', '2.96'],
       ['1.18', '4.94'],
       ['2.18', '1.7']], dtype='<U5')

In [7]:
ar = np.concatenate([names, coefs], axis=1)
ar 

array([['Крис Барнетт', 'Чейс Шерман', '2.748', '1.45'],
       ['Джеральд Миршерт', 'Джозеф Пайфер', '2.464', '1.54'],
       ['Игнасио Багамондес', 'Николас Мотта', '1.372', '3.015'],
       ['Келвин Гастелум', 'Крис Кертис', '1.83', '1.94'],
       ['Стив Гарсия-младший', 'Шайлан Нурданбек', '2.69', '1.46'],
       ['Сэм Хьюз', 'Жаклин Аморим', '2.976', '1.39'],
       ['Мишель Уотерсон', 'Луана Пиньейро', '2.43', '1.552'],
       ['Синтия Кальвильо', 'Люпита Годинез', '3.14', '1.37'],
       ['Роб Фонт', 'Адриан Янез', '2.348', '1.59'],
       ['Кевин Холланд', 'Сантьяго Понзиниббио', '1.39', '2.96'],
       ['Гилберт Бернс', 'Хорхе Масвидал', '1.18', '4.94'],
       ['Алекс Перейра', 'Исраэль Адесаня', '2.18', '1.7']], dtype='<U20')

In [8]:

fighter_coef_df = pd.DataFrame(ar, columns=['fighter1', 'fighter2', 'coef1', 'coef2'])
fighter_coef_df

,fighter1,fighter2,coef1,coef2
0,Крис Барнетт,Чейс Шерман,2.748,1.45
1,Джеральд Миршерт,Джозеф Пайфер,2.464,1.54
2,Игнасио Багамондес,Николас Мотта,1.372,3.015
3,Келвин Гастелум,Крис Кертис,1.83,1.94
4,Стив Гарсия-младший,Шайлан Нурданбек,2.69,1.46
5,Сэм Хьюз,Жаклин Аморим,2.976,1.39
6,Мишель Уотерсон,Луана Пиньейро,2.43,1.552
7,Синтия Кальвильо,Люпита Годинез,3.14,1.37
8,Роб Фонт,Адриан Янез,2.348,1.59
9,Кевин Холланд,Сантьяго Понзиниббио,1.39,2.96


In [9]:
df_prev = pd.read_excel('coefs.xlsx')
# df_prev = pd.read_excel('coefs_ufc268.xlsx')



In [10]:
df_all = pd.concat([df_prev, fighter_coef_df], ignore_index=True)
df_all['coef1'] = df_all['coef1'].astype(float)
df_all['coef2'] = df_all['coef2'].astype(float)


In [11]:
df_all
# нули и пустоты удалить потом

,fighter1,fighter2,coef1,coef2
0,Карлос Вергара,Оде Осборн,2.600,1.52
1,Мелсик Багдасарян,Бруно Соуза,1.280,3.85
2,Дастин Джакоби,Джон Аллан Арте,1.250,4.15
3,Джан Вилланте,Крис Барнетт,1.780,2.08
4,Иан Гарри,Джордан Уильямс,1.250,4.10
...,...,...,...,...
451,Синтия Кальвильо,Люпита Годинез,3.140,1.37
452,Роб Фонт,Адриан Янез,2.348,1.59
453,Кевин Холланд,Сантьяго Понзиниббио,1.390,2.96
454,Гилберт Бернс,Хорхе Масвидал,1.180,4.94


In [12]:
df_all.to_excel('coefs.xlsx', index=False)

In [13]:
browser.quit()